In [328]:
# Import necessary libraries and modules
import pandas as pd
import numpy as np
import john_acquire as a  # Custom module for data acquisition
%load_ext autoreload
%autoreload 2

# Set the option to display all columns in DataFrames
pd.set_option('display.max_columns', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Dataset Overview

| Column | Description | Data Type |
| --- | --- | --- |
| camis | Unique identifier for each record | int64 |
| dba | Doing Business As (DBA) name | object |
| boro | Borough where the establishment is located | object |
| building | Building number | object |
| street | Street name | object |
| zipcode | Zip code | float64 |
| phone | Phone number | object |
| cuisine\_description | Description of the cuisine type | object |
| inspection\_date | Date of inspection | object |
| action | Action taken during inspection | object |
| critical\_flag | Indicator of critical violations | object |
| score | Inspection score | float64 |
| record\_date | Date of record | object |
| inspection\_type | Type of inspection | object |
| latitude | Latitude of the establishment | float64 |
| longitude | Longitude of the establishment | float64 |
| community\_board | Community board district | float64 |
| council\_district | Council district | float64 |
| census\_tract | Census tract | float64 |
| bin | Building identification number | float64 |
| bbl | Borough block and lot number | float64 |
| nta | Neighborhood Tabulation Area (NTA) | object |
| violation\_code | Code indicating violations | object |
| violation\_description | Description of the violations | object |
| grade | Inspection grade | object |
| grade\_date | Date of inspection grade | object |

In [329]:
# Load the acquired data from the CSV file
inspection_df = pd.read_csv('nyc_health_inspections_2000_to_2023.csv', index_col=False)

# Display the first few rows of the loaded DataFrame
inspection_df.head()

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
0,50067297,GERBASI RESTAURANT,Bronx,2389,ARTHUR AVENUE,10458.0,7182205735,Italian,2021-09-12T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.855290,-73.887797,206.0,15.0,39100.0,2011897.0,2.030650e+09,BX06,NaN,NaN,NaN,NaN
1,50034232,RELISH CATERERS,Bronx,2501,3 AVENUE,10451.0,2122281672,American,2021-09-25T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.810202,-73.928401,201.0,8.0,5100.0,2000795.0,2.023200e+09,BX39,NaN,NaN,NaN,NaN
2,50064240,DAXI SICHUAN,Queens,136-20,ROOSEVELT AVENUE,11354.0,9175631983,Chinese,2022-09-21T00:00:00.000,Violations were cited in the following area(s).,Not Critical,13.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.759778,-73.829235,407.0,20.0,85300.0,4113546.0,4.050190e+09,QN22,09B,Thawing procedure improper.,A,2022-09-21T00:00:00.000
3,50105603,LE PAIN QUOTIDIEN,Manhattan,81,WEST BROADWAY,10007.0,6468639168,French,2022-11-25T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Re-inspection,40.715083,-74.009567,101.0,1.0,2100.0,1001480.0,1.001368e+09,MN24,NaN,NaN,NaN,NaN
4,50069583,PHO BEST,Queens,4235,MAIN ST,11355.0,9173618878,Southeast Asian,2022-05-09T00:00:00.000,Violations were cited in the following area(s).,Critical,30.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.754418,-73.827881,407.0,20.0,85300.0,4573539.0,4.051358e+09,QN22,02B,Hot food item not held at or above 140º F.,NaN,NaN


In [330]:
inspections_df_status = pd.DataFrame({
    'Null Count': inspection_df.isna().sum(),
    'Zero Count': (inspection_df == 0).sum(),
    'Zero Count (str)': (inspection_df == '0').sum(),
    'Data Types': inspection_df.dtypes
})
# Name the index
inspections_df_status = inspections_df_status.rename_axis('Column')

# inspections_df_status.to_csv('inspections_df_status.csv')
ispections_prepare = inspections_df_status[inspections_df_status[['Null Count', 'Zero Count', 'Zero Count (str)']].gt(0).any(axis=1)]
# ispections_prepare.to_csv('ispections_prepare.csv')

ispections_prepare

,Null Count,Zero Count,Zero Count (str),Data Types
Column,,,,
building,289,0,430,object
zipcode,2642,0,0,float64
phone,6,0,0,object
score,7572,9813,0,float64
latitude,246,2642,0,float64
longitude,246,2642,0,float64
community_board,3153,0,0,float64
council_district,3157,0,0,float64
census_tract,3157,0,0,float64


In [331]:
# Display information about the dataset, including non null counts per column
inspection_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207365 entries, 0 to 207364
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  207365 non-null  int64  
 1   dba                    207365 non-null  object 
 2   boro                   207365 non-null  object 
 3   building               207076 non-null  object 
 4   street                 207365 non-null  object 
 5   zipcode                204723 non-null  float64
 6   phone                  207359 non-null  object 
 7   cuisine_description    207365 non-null  object 
 8   inspection_date        207365 non-null  object 
 9   action                 207365 non-null  object 
 10  critical_flag          207365 non-null  object 
 11  score                  199793 non-null  float64
 12  record_date            207365 non-null  object 
 13  inspection_type        207365 non-null  object 
 14  latitude               207119 non-nu

### Checking for Missing Values

Summarizing Missing Values by Column

In [332]:
inspections_df_isna = pd.DataFrame({
    'Null Count': inspection_df.isna().sum(),
})
# Name the index
inspections_df_isna = inspections_df_isna.rename_axis('Column')

inspections_df_isna_true = inspections_df_isna[inspections_df_isna['Null Count'] > 0]

inspections_df_isna_true
# inspections_df_isna_true.to_csv('inspections_df_isna.csv')

,Null Count
Column,
building,289
zipcode,2642
phone,6
score,7572
latitude,246
longitude,246
community_board,3153
council_district,3157
census_tract,3157


In [333]:
# Calculate the count of missing values in each column
null_counts_by_column = inspection_df.isnull().sum()

# Filter and display columns with missing values
null_counts_by_column[null_counts_by_column > 0]

building                    289
zipcode                    2642
phone                         6
score                      7572
latitude                    246
longitude                   246
community_board            3153
council_district           3157
census_tract               3157
bin                        4144
bbl                         511
nta                        3153
violation_code             1157
violation_description      1157
grade                    104404
grade_date               113157
dtype: int64

### Grades Column

Drop the 'grade' and 'grade\_date' columns. According to the documentation, not all inspections receive a grade. We can simply calculate the grade using the score. Additionally, the documentation mentions that the grade may not match the scores due to input errors.

In [334]:
# Dropping the 'grade' and 'grade_date' columns
inspection_df = inspection_df.drop(['grade', 'grade_date'], axis=1)

In [335]:
# Reassessing the null counts in the dataset
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

building                  289
zipcode                  2642
phone                       6
score                    7572
latitude                  246
longitude                 246
community_board          3153
council_district         3157
census_tract             3157
bin                      4144
bbl                       511
nta                      3153
violation_code           1157
violation_description    1157
dtype: int64

##### Inferring Missing Values

Our next step is to strategize how to address these missing values by leveraging available data in other columns. The proposed hierarchy for inference is as follows:

`lat&long < building < bin < bbl < nta, zipcode* < community board < council district < census tract`

Given the relatively low count of missing values in the BBL column, it appears to be a promising candidate for inferring related data such as NTA (Neighborhood Tabulation Area), Community Board, Council District, and Census Tract.

Let's examine the first few unique values in the BBL column to understand its content

In [336]:
# Let's take a look at the first few unique values in the BBL column
sorted(inspection_df.bbl.unique())[:10]


[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 1000010010.0,
 1000020001.0,
 1000047501.0,
 1000070028.0,
 1000070031.0]

The BBL column shows the presence of non-standard values, which do not conform to the expected 10-digit format (1.0, 2.0, 3.0, 4.0, etc)

Now, let's find out the count of these non-standard values:

In [337]:
# Define non-standard BBL values
bbl_values = [np.nan, 1.0, 2.0, 3.0, 4.0, 5.0]

# Calculate the count of these non-standard values in the BBL column
inspection_df['bbl'].isin(bbl_values).sum()

4144

Non-standard BBL values are exactly the same as NaN values in the BIN column, indicating a pattern of missing values across these key columns.

- census_tract               3157
- bin                        4144
- bbl                        4144
- nta                        3153
- community_board            3153
- council_district           3157

We are unable to rely on bbl make inferences because the features were missing across the same rows. We must abandon the hierarchy inference plan. To proceed, we drop rows with NaN values in the BIN column.

In [338]:
# Dropping rows with null values in the 'bin' column
inspection_df = inspection_df.dropna(subset=['bin'])


##### Reevaluating Null Counts After BIN Column Cleanup

In [339]:
# Calculate the count of missing values in each column
null_counts_by_column = inspection_df.isnull().sum()

# Filter and display columns with missing values
null_counts_by_column[null_counts_by_column > 0]

zipcode                    30
phone                       6
score                    7440
community_board            30
council_district           34
census_tract               34
nta                        30
violation_code           1076
violation_description    1076
dtype: int64

As expected, the remaining NaNs are mostly related or in common with the initial set. 

##### Handling Remaining Zoning Nulls

For the small number of remaining NaNs in the zoning columns, we can safely drop them due to their limited impact on the dataset. I chose to drop 'council_district' to see if this also got rid of the other NaNs.

In [340]:
# Dropping rows with null values in the 'council_district' column
inspection_df = inspection_df.dropna(subset=['council_district'])

# Reassessing the null counts in the dataset
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

phone                       6
score                    7438
violation_code           1076
violation_description    1076
dtype: int64

As expected, dropping council_district also removed the other zoning features with nulls.

##### Identifying Relevant Inspection Types

Before proceeding with the score nulls, let's identify and focus on inspection types related to food safety.

In [341]:
# Assuming 'inspection_df' is your DataFrame
unique_inspection_types = inspection_df['inspection_type'].unique()

# Convert the numpy array to a list and then sort it
sorted_inspection_types = sorted(unique_inspection_types.tolist())
sorted_inspection_types

['Administrative Miscellaneous / Compliance Inspection',
 'Administrative Miscellaneous / Initial Inspection',
 'Administrative Miscellaneous / Re-inspection',
 'Administrative Miscellaneous / Reopening Inspection',
 'Administrative Miscellaneous / Second Compliance Inspection',
 'Calorie Posting / Compliance Inspection',
 'Calorie Posting / Initial Inspection',
 'Calorie Posting / Re-inspection',
 'Cycle Inspection / Compliance Inspection',
 'Cycle Inspection / Initial Inspection',
 'Cycle Inspection / Re-inspection',
 'Cycle Inspection / Reopening Inspection',
 'Cycle Inspection / Second Compliance Inspection',
 'Inter-Agency Task Force / Initial Inspection',
 'Inter-Agency Task Force / Re-inspection',
 'Pre-permit (Non-operational) / Compliance Inspection',
 'Pre-permit (Non-operational) / Initial Inspection',
 'Pre-permit (Non-operational) / Re-inspection',
 'Pre-permit (Non-operational) / Second Compliance Inspection',
 'Pre-permit (Operational) / Compliance Inspection',
 'Pre-per

 We will exclude types such as "Calorie Posting," "Pre-permit," "Smoke-Free Air Act," and "Trans Fat," as they do not directly pertain to food safety

In [342]:
original_length = len(inspection_df)
 List of inspection types to be removed
remove_types = ["Calorie Posting", "Pre-permit", "Smoke-Free Air Act", "Trans Fat"]

# Filter the DataFrame in a single step
inspection_df = inspection_df[~inspection_df['inspection_type'].str.startswith(tuple(remove_types))]
new_length = len(inspection_df)
print(f' {original_length} - {new_length} = {(original_length - new_length)}')

 203187 - 155970 = 47217


In [343]:
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

phone                       6
score                    6024
violation_code            797
violation_description     797
dtype: int64

Eliminating these rows led to a modest decrease in null values, due to the overlap in missing data among these rows. However, a detailed analysis of a few outstanding violation codes is still required.

Now, let's examine the history of a restaurant with a null value in the violation code to understand the reasons behind this occurrence.

In [344]:
inspection_df[inspection_df['camis'] == 40365644]

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
99872,40365644,JOE ALLEN RESTAURANT,Manhattan,326,WEST 46 STREET,10036.0,2125816464,American,2022-09-20T00:00:00.000,Violations were cited in the following area(s).,Not Critical,12.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,40.760179,-73.988882,104.0,3.0,12100.0,1025016.0,1.010360e+09,MN15,10B,Anti-siphonage or back-flow prevention device ...
102373,40365644,JOE ALLEN RESTAURANT,Manhattan,326,WEST 46 STREET,10036.0,2125816464,American,2022-09-20T00:00:00.000,Violations were cited in the following area(s).,Critical,12.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,40.760179,-73.988882,104.0,3.0,12100.0,1025016.0,1.010360e+09,MN15,04H,"Raw, cooked or prepared food is adulterated, c..."
105048,40365644,JOE ALLEN RESTAURANT,Manhattan,326,WEST 46 STREET,10036.0,2125816464,American,2022-02-09T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:09.000,Cycle Inspection / Initial Inspection,40.760179,-73.988882,104.0,3.0,12100.0,1025016.0,1.010360e+09,MN15,NaN,NaN
110868,40365644,JOE ALLEN RESTAURANT,Manhattan,326,WEST 46 STREET,10036.0,2125816464,American,2022-09-20T00:00:00.000,Violations were cited in the following area(s).,Not Critical,12.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,40.760179,-73.988882,104.0,3.0,12100.0,1025016.0,1.010360e+09,MN15,10F,Non-food contact surface or equipment made of ...


In [345]:
# Group by 'camis' and 'inspection_date' and check for nulls in 'violation_code'
grouped = inspection_df.groupby(['camis', 'inspection_date'])
groups_with_nulls = grouped.apply(lambda x: x['violation_code'].isna().any())

# Count the number of rows in each group
group_sizes = grouped.size()

# Filter the DataFrame to include only those groups with nulls in 'violation_code' and at least 2 rows
filtered_groups = groups_with_nulls[groups_with_nulls].index.intersection(group_sizes[group_sizes >= 2].index)
filtered_df = inspection_df[inspection_df.set_index(['camis', 'inspection_date']).index.isin(filtered_groups)].reset_index(drop=True)

# Now, 'filtered_df' contains only the groups where there are null values in 'violation_code' and at least 2 rows in the group
filtered_df.sort_values(by='camis').head()

# inspections_group = filtered_df.sort_values(by='camis').head()
# inspections_group.to_csv('inspections_camisg.csv')

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
372,40390409,THE FAMOUS JIMBO'S HAMBURGER PALACE,Manhattan,1345,AMSTERDAM AVENUE,10027.0,2128658777,Hamburgers,2021-07-23T00:00:00.000,Violations were cited in the following area(s).,Critical,26.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.813705,-73.956012,109.0,7.0,20901.0,1084098.0,1.019660e+09,MN09,04M,Live roaches present in facility's food and/or...
199,40390409,THE FAMOUS JIMBO'S HAMBURGER PALACE,Manhattan,1345,AMSTERDAM AVENUE,10027.0,2128658777,Hamburgers,2021-07-23T00:00:00.000,Violations were cited in the following area(s).,Critical,26.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.813705,-73.956012,109.0,7.0,20901.0,1084098.0,1.019660e+09,MN09,02G,Cold food item held above 41º F (smoked fish a...
692,40390409,THE FAMOUS JIMBO'S HAMBURGER PALACE,Manhattan,1345,AMSTERDAM AVENUE,10027.0,2128658777,Hamburgers,2021-07-23T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Initial Inspection,40.813705,-73.956012,109.0,7.0,20901.0,1084098.0,1.019660e+09,MN09,NaN,NaN
431,40390409,THE FAMOUS JIMBO'S HAMBURGER PALACE,Manhattan,1345,AMSTERDAM AVENUE,10027.0,2128658777,Hamburgers,2021-07-23T00:00:00.000,Violations were cited in the following area(s).,Critical,26.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.813705,-73.956012,109.0,7.0,20901.0,1084098.0,1.019660e+09,MN09,06F,Wiping cloths soiled or not stored in sanitizi...
169,40390409,THE FAMOUS JIMBO'S HAMBURGER PALACE,Manhattan,1345,AMSTERDAM AVENUE,10027.0,2128658777,Hamburgers,2021-07-23T00:00:00.000,Violations were cited in the following area(s).,Not Critical,26.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.813705,-73.956012,109.0,7.0,20901.0,1084098.0,1.019660e+09,MN09,08A,Facility not vermin proof. Harborage or condit...


The data format clearly shows that individual violations from an inspection are recorded on separate rows. However, it's unusual to observe different types of inspections, such as 'Administrative Miscellaneous', occurring concurrently within a single visit. Notably, rows categorized under 'Administrative Miscellaneous' frequently present missing data in the 'score, 'violation_code' and 'violation_description' fields. The next step in our analysis involves determining the frequency of NaN values within the 'Administrative Miscellaneous' inspection category

In [346]:
# Group by 'inspection_type' and count null 'violation_code' entries
null_score_count = inspection_df.groupby('inspection_type').apply(lambda x: x['score'].isnull().sum())

# The result is a Series where the index is 'inspection_type' and the values are the counts of null 'violation_code'
print(null_score_count)


inspection_type
Administrative Miscellaneous / Compliance Inspection             99
Administrative Miscellaneous / Initial Inspection              4899
Administrative Miscellaneous / Re-inspection                    975
Administrative Miscellaneous / Reopening Inspection              43
Administrative Miscellaneous / Second Compliance Inspection       8
Cycle Inspection / Compliance Inspection                          0
Cycle Inspection / Initial Inspection                             0
Cycle Inspection / Re-inspection                                  0
Cycle Inspection / Reopening Inspection                           0
Cycle Inspection / Second Compliance Inspection                   0
Inter-Agency Task Force / Initial Inspection                      0
Inter-Agency Task Force / Re-inspection                           0
dtype: int64


The analysis reveals that all the missing 'score' rows are tied to various "Administrative" inspection types. This pattern suggests that "Administrative" inspections might be documenting a distinct category of violations, particularly given the occurrence of both "Administrative" and "Cycle" inspections within the same visit. This finding indicates a potential strategy to either deduce the score for these cases or consider the removal of "Administrative" inspection types from our dataset.

To proceed effectively, it's important to closely examine the specific types of violations recorded under "Administrative" inspections. Understanding the nuances of these violations will assist in determining their relevance to our overall data analysis and their impact on the comprehensive scoring system.

We filter the dataset to only include rows where 'inspection_type' starts with "Administrative" and then identified the unique 'violation_description' values to understand the nature of violations in "Administrative" inspections.

In [347]:
# Filter for rows where 'inspection_type' starts with "Administrative"
administrative_rows = inspection_df[inspection_df['inspection_type'].str.startswith("Administrative")]

# Get a count of each unique 'violation_description' in these rows
violation_description_counts = administrative_rows['violation_description'].value_counts()

# Display the counts
violation_description_counts

violation_description
“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted.                                                                                                                                                 763
Food allergy information poster not conspicuously posted where food is being prepared or processed by food workers.                                                                                                                                                                                                                                  699
Current letter grade or Grade Pending card not posted                                                                                                                                                                                                                                           

The analysis of "Administrative" inspections revealed that these primarily involve non-food safety violations, such as missing posters, signage, or documentation, rather than critical food safety issues. Common violations in "Administrative" inspections include:

- Missing "Choking first aid" and "Alcohol and pregnancy" posters.
- Failure to post or conspicuously post current letter grades or Grade Pending cards.
- Providing certain items without customer request, such as plastic straws.

Given that "Administrative" inspections do not contribute to our food safety analysis and mainly involve non-critical violations, we made the decision to drop rows where the 'inspection_type' starts with "Administrative." This step helps streamline the dataset and focuses our analysis on relevant food safety factors.

In [348]:
# Drop rows where 'inspection_type' starts with "Administrative"
inspection_df = inspection_df[~inspection_df['inspection_type'].str.startswith("Administrative")]
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

phone                      6
violation_code           438
violation_description    438
dtype: int64

As a result, the null values in the 'score' column have been successfully addressed, leaving no NaNs in this column. Moving forward, we will further investigate the remaining null values in the 'violation_code' and 'violation_description' columns to gain insights into their presence, even though their frequency is relatively low.

In [349]:
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = inspection_df.groupby('inspection_type').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count

inspection_type
Cycle Inspection / Compliance Inspection             2
Cycle Inspection / Initial Inspection              256
Cycle Inspection / Re-inspection                    40
Cycle Inspection / Reopening Inspection             70
Cycle Inspection / Second Compliance Inspection      0
Inter-Agency Task Force / Initial Inspection        69
Inter-Agency Task Force / Re-inspection              1
dtype: int64

As observed, the presence of null values in the 'violation_code' and 'violation_description' columns varies depending on the inspection type. While this insight doesn't directly explain why these nulls exist, it's a useful observation. To further investigate the underlying reasons behind these null values, we can analyze the 'action' column, which may provide more context.

In [350]:
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count

action
Establishment re-opened by DOHMH.                               70
No violations were recorded at the time of this inspection.    364
Violations were cited in the following area(s).                  4
dtype: int64

In [351]:
null_violation_count.sum()

438

Our analysis has revealed that a significant portion of the null values in the 'violation_code' and 'violation_description' columns are associated with inspections where no violations were recorded. To address this, we plan to replace these null values for the 'violation_code' column with "none" and for the 'violation_description' column with "No violations were recorded." 

In [352]:
# Identify rows where 'action' starts with the specified strings and 'violation_code' is null
condition = inspection_df['violation_code'].isna() & inspection_df['action'].str.startswith("No violations were recorded at the time of this inspection.")

# Update 'violation_code' and 'violation_description' for these rows
inspection_df.loc[condition, ['violation_code', 'violation_description']] = ['none', 'No violations were recorded']

In [353]:
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count

action
Establishment re-opened by DOHMH.                  70
Violations were cited in the following area(s).     4
dtype: int64

To gain further clarity and address the remaining null values in the 'violation_code' and 'violation_description' columns, we will focus on a subset of rows related to reopening inspections. Specifically, we will examine these rows to understand why some of them have null values in these columns.

In [354]:
# Filter rows where 'inspection_type' starts with "Administrative"
action_reopened = inspection_df[inspection_df['action'].str.startswith("Establishment re-opened by DOHMH")]
action_reopened.head(10)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
9,50078860,A FEI CHINESE RESTAURANT,Brooklyn,553,THROOP AVENUE,11216.0,7184535205,Chinese,2022-02-23T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.683160,-73.940966,303.0,36.0,27500.0,3052864.0,3.018410e+09,BK35,NaN,NaN
88,50032753,AGAVI ORGANIC JUICEBAR,Manhattan,72,EAST 7 STREET,10003.0,2123908042,"Juice, Smoothies, Fruit Salads",2022-03-11T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.727390,-73.986766,103.0,2.0,3800.0,1006277.0,1.004480e+09,MN22,NaN,NaN
139,50005590,FAMOUS SICHUAN,Manhattan,10,PELL STREET,10013.0,2122333888,Chinese,2022-09-29T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.714729,-73.997598,103.0,1.0,2900.0,1001776.0,1.001630e+09,MN27,NaN,NaN
227,41549281,DUNKIN,Manhattan,316,WEST 34 STREET,10001.0,2127602600,Donuts,2022-07-29T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.752494,-73.994221,104.0,3.0,10300.0,1013552.0,1.007570e+09,MN13,NaN,NaN
249,41642570,JOHN'S DELI,Brooklyn,2438,STILLWELL AVENUE,11223.0,7187144377,American,2022-01-25T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.588029,-73.983622,313.0,47.0,30800.0,3187046.0,3.069050e+09,BK26,NaN,NaN
311,50093419,SUNNY SUSHI & POKE,Manhattan,1457,3 AVENUE,10028.0,2125850300,Japanese,2023-04-06T00:00:00.000,Establishment re-opened by DOHMH.,Critical,12.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.776434,-73.955709,108.0,5.0,13800.0,1081259.0,1.015280e+09,MN32,04M,Live roaches in facility's food or non-food area.
694,50103925,CAFE GOSSIP,Brooklyn,2338,CONEY ISLAND AVENUE,11223.0,7185766233,Turkish,2023-04-05T00:00:00.000,Establishment re-opened by DOHMH.,Not Critical,14.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.599842,-73.961318,315.0,48.0,39400.0,3192187.0,3.071120e+09,BK25,10B,Anti-siphonage or back-flow prevention device ...
843,50093318,Lamia's Fish Market,Manhattan,45,AVENUE B,10009.0,2127773650,Seafood,2023-02-02T00:00:00.000,Establishment re-opened by DOHMH.,Not Critical,3.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.722965,-73.982734,103.0,2.0,2601.0,1076894.0,1.003860e+09,MN28,10F,Non-food contact surface or equipment made of ...
1010,50041309,OZU JAPANESE CUISINE & LOUNGE,Brooklyn,78,CLARK STREET,11201.0,7187973288,Japanese,2023-02-01T00:00:00.000,Establishment re-opened by DOHMH.,Not Critical,3.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.697527,-73.993317,302.0,33.0,501.0,3001864.0,3.002360e+09,BK09,10F,Non-food contact surface or equipment made of ...
1148,50067048,NEW ORIENTAL HOUSE,Bronx,565,EAST 184 STREET,10458.0,7185638888,Chinese,2023-10-16T00:00:00.000,Establishment re-opened by DOHMH.,Critical,6.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.854508,-73.890242,206.0,15.0,38700.0,2011674.0,2.030540e+09,BX06,06C,"Food, supplies, or equipment not protected fro..."


In [355]:
# action_reopened.head().to_csv('action_reopened.csv')

In the case of reopening inspections, we observed that some rows had NaN values in the violation code/description, while others had codes and descriptions, suggesting the absence of violations. Additionally, the "critical_flag" column contained 'Not Applicable' when no violations were present. We can reasonably assume that this indicates no violations were found during those inspections. Therefore, we will be replacing violation_code and violation_description NaNs with 'none' and 'No violations were recorded', respectively.

In [356]:
# Identify rows where 'action' starts with the specified strings and 'violation_code' is null
condition = (inspection_df['violation_code'].isna() & 
            inspection_df['action'].str.startswith("Establishment re-opened") &
           ( inspection_df['critical_flag'] == 'Not Applicable'))

# Update 'violation_code' and 'violation_description' for these rows
inspection_df.loc[condition, ['violation_code', 'violation_description']] = ['none', 'No violations were recorded']

In [357]:
# Create a DataFrame containing rows where 'violation_code' is null
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]

# Group the DataFrame by 'action' and count null 'violation_code' entries for each group
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())

# Display the count of null 'violation_code' entries for each 'action'
null_violation_count

action
Violations were cited in the following area(s).    4
dtype: int64

That leaves us with:
- Violations were cited in the following area(s).


In [358]:
# Filter rows where 'inspection_type' doesn't start with "Violations"
action_violationcited = inspection_df[~inspection_df['inspection_type'].str.startswith("Violations")]

# Sort the DataFrame by 'violation_code' with NaN values at the beginning
action_violationcited = action_violationcited.sort_values(by='violation_code', na_position='first')

action_violationcited.head()


,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
36548,41688093,PAISANOS BURGERS\MELT,Brooklyn,620,ATLANTIC AVENUE,11217.0,9176186310,Hamburgers,2018-05-11T00:00:00.000,Violations were cited in the following area(s).,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN,NaN
69283,41564956,RADIO CITY MUSIC HALL,Manhattan,1260,AVENUE OF THE AMERICAS,10020.0,2124857000,American,2017-08-09T00:00:00.000,Violations were cited in the following area(s).,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.759983,-73.980349,105.0,4.0,10400.0,1083861.0,1.012660e+09,MN17,NaN,NaN
72054,41644180,MANHATTAN TERRACE BAR,Queens,110-00,ROCKAWAY BOULEVARD,11420.0,2122153542,American,2016-05-14T00:00:00.000,Violations were cited in the following area(s).,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.677665,-73.828758,410.0,32.0,86400.0,4457718.0,4.115430e+09,QN55,NaN,NaN
142475,41688142,TABLE 87,Brooklyn,620,ATLANTIC AVENUE,11217.0,9176186100,Pizza,2018-05-11T00:00:00.000,Violations were cited in the following area(s).,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN,NaN
38387,50078693,ZENITH THAI,Queens,7002,FRESH POND RD,11385.0,7184978899,Thai,2023-02-22T00:00:00.000,Violations were cited in the following area(s).,Critical,26.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,40.702469,-73.895174,405.0,30.0,58300.0,4085294.0,4.035370e+09,QN20,02A,Time/Temperature Control for Safety (TCS) food...


In [359]:
# action_violationcited.head().to_csv('action_violationcited.csv')

We examined inspections with the action "Violations were cited in the following area(s)" which had a mix of nulls and codes in the violation_code column. Since we cannot determine what the code should be for these cases, we have made the decision to drop these rows.

In [360]:
inspection_df = inspection_df.drop(inspection_df[(inspection_df['violation_code'].isna()) &
                                                 (inspection_df['action'].str.startswith("Violations were cited in the following area(s)"))].index)

### Phone

There are only a few rows with nulls in this column. We can fill these remaining nulls with a common placeholder, such as '0000000000,':

In [361]:
# Fill remaining nulls in numerical columns with '0000000000'
inspection_df['phone'].fillna('0000000000', inplace=True)

# Reassessing the null counts in the dataset
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

Series([], dtype: int64)

There are no violation_code nulls left.

In [362]:
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

Series([], dtype: int64)

We have successfully addressed all the nulls in the DataFrame. 

In [366]:
null_zero_counts = pd.DataFrame({
    'Numeric_Zero_Count': (inspection_df == 0).sum(),
    'String_Zero_Count': (inspection_df == '0').sum(),
    'Null_Count': (inspection_df.isna().sum()).sum(),
    'Blank Count': (inspection_df == '').sum(),
    'Space Count': (inspection_df == ' ').sum(),
    'Data Types': inspection_df.dtypes
})

null_zero_counts

# null_zero_counts.to_csv('null_zero_counts.csv')

#### Dealing with 0s

##### Building

For the 'building' column, it appears to have some 0 values, but there's not much we can do about that, so we will leave it as is.

##### Score
Regarding the 'score' column, we can infer that a score of 0 indicates no violations.

### Dealing with Data Types

In [81]:
inspection_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149942 entries, 0 to 207364
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  149942 non-null  int64  
 1   dba                    149942 non-null  object 
 2   boro                   149942 non-null  object 
 3   building               149942 non-null  object 
 4   street                 149942 non-null  object 
 5   zipcode                149942 non-null  float64
 6   phone                  149942 non-null  object 
 7   cuisine_description    149942 non-null  object 
 8   inspection_date        149942 non-null  object 
 9   action                 149942 non-null  object 
 10  critical_flag          149942 non-null  object 
 11  score                  149942 non-null  float64
 12  record_date            149942 non-null  object 
 13  inspection_type        149942 non-null  object 
 14  latitude               149942 non-null  f

### Building Column
First, lets address the building column.

In [82]:
inspection_df['building'].str.isalpha().any()

True

Building has a mix of letter and numbers, it must remain an object type. 

### Score Column

Prepare the 'score' column for numerical analysis, the following action has been taken.

In [83]:
inspection_df['score'] = inspection_df['score'].astype(int)

### Float data type columns

The following columns should exclusively contain whole numbers. Currently, they are in float type. To ensure their integrity:

1. Verify if they consist of whole numbers.
2. Convert them to integers to confirm the absence of special characters.
3. Convert them back to strings, as these columns are categorical features.

In [84]:
columns_to_check = ['zipcode', 'score', 'community_board', 'council_district', 'census_tract', 'bin', 'bbl']

for column in columns_to_check:
    is_integer = (inspection_df[column] % 1 == 0).all()
    print(f"{column} Column: {is_integer}")

zipcode Column: True
score Column: True
community_board Column: True
council_district Column: True
census_tract Column: True
bin Column: True
bbl Column: True


In [85]:
for column in columns_to_check:
    inspection_df[column] = inspection_df[column].astype(int)
    inspection_df[column] = inspection_df[column].astype(str)

### Phone Column

Lets work on the 'phone' column, we will perform the following steps:

1. Remove all non-numerical characters from the 'phone' column.
2. Replace missing or empty values with '1000000000' to avoid having all zeros.


In [86]:
# Use regex to extract digits from the "phone" column
inspection_df['phone'] = inspection_df['phone'].str.replace(r'\D', '', regex=True)

In [87]:
# Remove black or 0s placeholder with '1000000000'.
inspection_df['phone'] = inspection_df['phone'].str.strip().replace(['', '0000000000'], '1000000000')

## Inspection Date Column

To standardize the 'inspection_date' column, we will follow these steps:

1. Begin by printing the 'inspection_date' from the first row of the DataFrame to verify the initial format, which is in the format 'YYYY-MM-DDThh:mm:ss.sss'.
2. Next, convert the 'inspection_date' column to datetime format and format it to display only the date in 'YYYY-MM-DD' format.
3. Finally, print the 'inspection_date' from the first row of the DataFrame again to confirm that it has been standardized to 'YYYY-MM-DD'.



In [88]:
# Print the 'inspection_date' from the first row of the DataFrame
inspection_df.loc[0, 'inspection_date']

'2021-09-12T00:00:00.000'

In [89]:
# Convert the 'inspection_date' column to datetime and format it to display only the date (YYYY-MM-DD)
inspection_df['inspection_date'] = pd.to_datetime(inspection_df['inspection_date']).dt.strftime('%Y-%m-%d')

# Print the 'inspection_date' from the first row of the DataFrame
inspection_df.loc[0, 'inspection_date']


The DataFrame 'inspection_df' has been thoroughly checked and cleaned, resulting in the following characteristics:

- No null values exist in any of the columns.
- The data types of the columns are appropriate.

The data is now ready for further analysis and exploration. If you have any additional tasks or questions related to this DataFrame or any other topic, please feel free to ask.

In [91]:
inspection_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149942 entries, 0 to 207364
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  149942 non-null  int64  
 1   dba                    149942 non-null  object 
 2   boro                   149942 non-null  object 
 3   building               149942 non-null  object 
 4   street                 149942 non-null  object 
 5   zipcode                149942 non-null  object 
 6   phone                  149942 non-null  object 
 7   cuisine_description    149942 non-null  object 
 8   inspection_date        149942 non-null  object 
 9   action                 149942 non-null  object 
 10  critical_flag          149942 non-null  object 
 11  score                  149942 non-null  object 
 12  record_date            149942 non-null  object 
 13  inspection_type        149942 non-null  object 
 14  latitude               149942 non-null  f